In [1]:
# ------------------------------------------------------------
# 1. Imports
# ------------------------------------------------------------
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import tensorflow as tf


In [2]:

# ------------------------------------------------------------
# 2. Chemins des fichiers
# ------------------------------------------------------------
BASELINE_HISTORY_PATH = "../models/history_baseline.pkl"
FINETUNE_HISTORY_PATH = "../models/history_finetune.pkl"
MODEL_PATH = "../models/inception_finetuned.h5"
REPORTS_DIR = "../reports/"

os.makedirs(REPORTS_DIR, exist_ok=True)

print("Dossiers vérifiés.")


Dossiers vérifiés.


In [7]:

# ------------------------------------------------------------
# 3. Charger les historiques
# ------------------------------------------------------------
with open(BASELINE_HISTORY_PATH, "rb") as f:
    history_baseline = pickle.load(f)

with open(FINETUNE_HISTORY_PATH, "rb") as f:
    history_finetune = pickle.load(f)

print("Historiques chargés.")


FileNotFoundError: [Errno 2] No such file or directory: '../models/history_baseline.pkl'

In [5]:

# ------------------------------------------------------------
# 4. Tracer les courbes d'apprentissage
# ------------------------------------------------------------

def plot_history(history, title, filename):
    plt.figure(figsize=(10, 5))
    
    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history["accuracy"], label="train")
    plt.plot(history["val_accuracy"], label="val")
    plt.title("Accuracy - " + title)
    plt.legend()
    
    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history["loss"], label="train")
    plt.plot(history["val_loss"], label="val")
    plt.title("Loss - " + title)
    plt.legend()
    
    plt.tight_layout()
    save_path = os.path.join(REPORTS_DIR, filename)
    plt.savefig(save_path)
    plt.close()
    print(f"Image sauvegardée : {save_path}")

plot_history(history_baseline, "Baseline", "baseline_training_history.png")
plot_history(history_finetune, "Fine-Tuning", "fine_tuning_history.png")


NameError: name 'history_baseline' is not defined

In [6]:

# ------------------------------------------------------------
# 5. Charger le modèle final
# ------------------------------------------------------------
model = tf.keras.models.load_model(MODEL_PATH)
print("Modèle final chargé.")


Modèle final chargé.


In [ ]:

# ------------------------------------------------------------
# 6. Générer la matrice de confusion
# ------------------------------------------------------------

DATASET_PATH = "../data/flower_images"
IMAGE_SIZE = (299, 299)
BATCH_SIZE = 16

datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_gen = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

y_true = test_gen.classes
class_names = list(test_gen.class_indices.keys())

y_pred = model.predict(test_gen)
y_pred_classes = np.argmax(y_pred, axis=1)

cm = confusion_matrix(y_true, y_pred_classes)

# Tracer la matrice
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_names,
            yticklabels=class_names)
plt.title("Matrice de Confusion")
plt.xlabel("Prédictions")
plt.ylabel("Réel")

cm_path = os.path.join(REPORTS_DIR, "confusion_matrix.png")
plt.savefig(cm_path)
plt.close()
print("Image sauvegardée :", cm_path)


In [ ]:

# ------------------------------------------------------------
# 7. Rapport de classification
# ------------------------------------------------------------

report = classification_report(y_true, y_pred_classes, target_names=class_names)

report_path = os.path.join(REPORTS_DIR, "classification_report.txt")
with open(report_path, "w") as f:
    f.write(report)

print("\nRapport de classification :")
print(report)
print("\nFichier enregistré :", report_path)

print("\n🎉 Évaluation terminée avec succès !")